In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from spindle_tracker.spatial import contiguous_regions
from spindle_tracker.trajectories import Trajectories

In [2]:
def distance(x1, y1, x2, y2) : 
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

In [3]:
# Code adapte d'Hadrien

def get_directions(traj, window, base_score, side, second=True, min_duration=5, t0=0):
    """
    """
    window = int(np.round(window , 1))

    smooth_traj = pd.rolling_mean(traj, window)

    raw_direction = (np.diff(smooth_traj) > 0)
    direction = np.array(['NN'] * len(smooth_traj))

    def f(x):
        return (x.sum() - (len(x) - x.sum())) / (2 * window - 1)
    scores = pd.rolling_apply(raw_direction, window, f)

    if side == 1:
        direction[scores > base_score] = 'AP'
        direction[scores < - base_score] = 'P'
    else:
        direction[scores > base_score] = 'P'
        direction[scores < - base_score] = 'AP'

    p = (contiguous_regions(direction == 'P') - window)
    ap = (contiguous_regions(direction == 'AP') - window)

    # Switch to second


    p += t0
    ap += t0

    p = p[((p[:, 1] - p[:, 0]) > min_duration)]
    ap = ap[((ap[:, 1] - ap[:, 0]) > min_duration)]

    # Re switch to indexes


    return p, ap, direction



def get_coherence(data_solid, base_score=0.15):
    """
    N - N : 0
    N - P : 1
    N - AP : 2
    AP - P : 3
    P - P : 4
    AP - AP : 5
    """
    
    idx = pd.IndexSlice
    ktA = data_solid["x.2"]
    ktB = data_solid["x.3"]

    _, _, directionsA = get_directions(ktA, window=10, base_score=base_score, side=-1,
                                            second=False, t0=0)
    _, _, directionsB = get_directions(ktB, window=10, base_score=base_score, side=1,
                                            second=False, t0=0)

    m = np.char.add(directionsA, directionsB)

    # N - N
    m[(m == 'NNNN')] = 0

    # N - P
    m[(m == 'NNP') | (m == 'PNN')] = 1

    # N - AP
    m[(m == 'NNAP') | (m == 'APNN')] = 2

    # AP - P
    m[(m == 'APP') | (m == 'PAP')] = 3

    # P - P
    m[(m == 'PP')] = 4

    # AP - AP
    m[(m == 'APAP')] = 5

    coherence = m.astype('int')
    return coherence

In [4]:
def get_color (n=6, cmap="hsv", alpha=None, rgba=False):
    import matplotlib.pyplot as plt
    cmap = plt.cm.get_cmap(name=cmap)

    
    ite = zip(np.linspace(0, 0.9, n), [0,1,2,3,4,5])
    colors = {label: cmap(i, alpha=alpha) for i, label in ite}

    if not rgba:
        def get_hex(rgba):
            rgba = np.round(np.array(rgba) * 255).astype('int')
            if not alpha:
                rgba = rgba[:3]
            return "#" + "".join(['{:02X}'.format(a) for a in rgba])

        colors = {label: get_hex(color) for label, color in colors.items()}

    return colors



def kymo(data_solid, use_interpolate=False, time_in_minutes=False, mpl_params={'ls': '-', 'marker': 'o'}, ax=None):
    """
    Print a kymograph of DataFrame of peaks (self.peaks_real{'x_proj']})
    """


    if use_interpolate:
        peaks = self.peaks_real_interpolated
        times = self.times_interpolated
    else:
        #peaks = self.peaks_real
        times = data_solid["time"]

    colors = list(get_color().values())

    import matplotlib.pyplot as plt

    if not ax:
        fig = plt.figure(figsize=(12, 7))
        ax = plt.subplot(111)
    else:
        fig = ax.get_figure()

    drawer = ax.plot

    #gps = peaks.groupby(level=['main_label', 'side']).groups

    spb1x = data_solid["x"]
    spb1y = data_solid["y"]
    spb2x = data_solid["x.1"]
    spb2y = data_solid["y.1"]
    kt1x = data_solid["x.2"]
    kt1y = data_solid["y.2"]
    kt2x = data_solid["x.3"]
    kt2y = data_solid["y.3"]
    spindleCenterx = (spb1x-spb2x)/2
    spindleCentery = (spb1y-spb2y)/2
    
    dist_spb1 = distance(spb1x,spb1y, spindleCenterx, spindleCentery)-1
    dist_spb2 = distance(spb2x,spb2y, spindleCenterx, spindleCentery)-1
    dist_kt1 = distance(kt1x,kt1y, spindleCenterx, spindleCentery)-1
    dist_kt2 = distance(kt2x,kt2y, spindleCenterx, spindleCentery)-1
    
    # Draw SPB
    x = ktA = dist_spb1
    drawer(times, x, label="SPB A", color=colors[0], **mpl_params)

    x = ktA = dist_spb2
    drawer(times, x, label="SPB B", color=colors[1], **mpl_params)

    # Draw Kt
    x = ktA = dist_kt1
    drawer(times, x, label="Kt A", color=colors[2], **mpl_params)

    x =ktA = dist_kt2
    drawer(times, x, label="Kt B", color=colors[3], **mpl_params)

    # Set axis limit
    ax.set_xlim(min(times), max(times))
    #m = np.abs(peaks['x_proj'].max())
    #ax.set_ylim(-m, m)
    ax.set_ylim(-1.25, 1.25)

    fontsize = 22

    if time_in_minutes:
        import matplotlib

        majorLocator = matplotlib.ticker.MultipleLocator(60)
        minorLocator = matplotlib.ticker.MultipleLocator(60)
        ax.xaxis.set_major_locator(majorLocator)
        ax.xaxis.set_minor_locator(minorLocator)

        majorFormatter = matplotlib.ticker.FuncFormatter(lambda x, y: "%.0f" % (x / 60.))
        ax.xaxis.set_major_formatter(majorFormatter)

        ax.set_xlabel('Time (mn)', fontsize=fontsize)

    else:
        ax.set_xlabel('Time (seconds)', fontsize=fontsize)

    ax.set_title("Kymograph like plot", fontsize=fontsize)
    ax.set_ylabel('Distance (µm)', fontsize=fontsize)


    #leg = ax.legend(loc='best', fancybox=True)
    #leg.get_frame().set_alpha(0.5)

    plt.grid(True)
    plt.tight_layout()

    return fig

In [5]:
def kymo_directions(data_solid, sister=True, base_score=0.15):
    """
    """


    times = data_solid["time"]

    idx = pd.IndexSlice
    spb1x = data_solid["x"]
    spb1y = data_solid["y"]
    spb2x = data_solid["x.1"]
    spb2y = data_solid["y.1"]
    kt1x = data_solid["x.2"]
    kt1y = data_solid["y.2"]
    kt2x = data_solid["x.3"]
    kt2y = data_solid["y.3"]
    spindleCenterx = (spb1x-spb2x)/2
    spindleCentery = (spb1y-spb2y)/2
    
    spbB = distance(spb1x,spb1y, spindleCenterx, spindleCentery)-1
    spbA = distance(spb2x,spb2y, spindleCenterx, spindleCentery)-1
    ktB = distance(kt1x,kt1y, spindleCenterx, spindleCentery)-1
    ktA = distance(kt2x,kt2y, spindleCenterx, spindleCentery)-1

    kts_traj = (ktA.values + ktB.values) / 2

    if sister:

        def color_run_traj(ax, times, traj, run_indexes, color, top=True):
            """
            """

            for t1, t2 in run_indexes:
                x = times[t1:t2]
                y1 = traj.values[t1:t2]
                if top:
                    y2 = [10] * len(y1)
                else:
                    y2 = [-10] * len(y1)
                ax.fill_between(x, y1, y2, alpha=0.15, color=color)

        fig = kymo(data_solid,mpl_params={'ls': '-', 'marker': ''})
        ax = fig.get_axes()[0]

        
        p, ap, _ = get_directions(ktA, window=20, base_score=base_score, side=-1,
                                       second=False, t0=0)
        color_run_traj(ax, times, ktA, p, 'g', top=True)
        color_run_traj(ax, times, ktA, ap, 'b', top=True)

        p, ap, _ = get_directions(ktB, window=20, base_score=base_score, side=1,
                                       second=False, t0=0)
        color_run_traj(ax, times, ktB, p, 'g', top=False)
        color_run_traj(ax, times, ktB, ap, 'b', top=False)

        
    else:

        p, ap, _ = get_directions(kts_traj, window=20, base_score=base_score, side=1,
                                       second=True, t0=times[0])

        fig = kymo(data_solid,mpl_params={'ls': '-', 'marker': ''})
        ax = fig.get_axes()[0]

        ax.plot(times, kts_traj, color='black')

        for t1, t2 in p:
            ax.axvspan(t1, t2, facecolor='g', alpha=0.15)

        for t1, t2 in ap:
            ax.axvspan(t1, t2, facecolor='b', alpha=0.15)

    return fig

In [6]:
# Changement de dossier
# Changement de dossier linux
#os.chdir ("/home/sophie/Documents/ResultatsCytosim/oscillations/proba_asymetrique/res/metaphase1kt_100MTseuil_20cohesinStiffness_160KTMTstiffness/run4/")


# Changement de dossier Windows
#os.chdir("G:/asymetrie/metaphase1kt_100MTseuil_20cohesinStiffness_60KTMTstiffness")

In [6]:
# Changement de dossier
#os.chdir ("/home/sophie/Documents/ResultatsCytosim/oscillations/proba_asymetrique/"\
#          "res/metaphase1kt_100MTseuil_20cohesinStiffness_160KTMTstiffness/run4/")


#os.chdir("G:/asymetrie/metaphase1kt_100MTseuil_20cohesinStiffness_140KTMTstiffness/run11/")

#os.chdir ("/media/sophie/Sophie/Film_Cytosim/Recapture/run4")
os.chdir ("/media/sophie/Sophie/Resultats_simulations/oscillations/proba_asymetrique/res/metaphase1kt_100MTseuil_20cohesinStiffness_140KTMTstiffness/run11")


#Ouverture des fichiers
solid  = open("solid.txt")
data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
solid.close()

kt1x = data_solid["x.2"]
kt1y = data_solid["y.2"]
kt2x = data_solid["x.3"]
kt2y = data_solid["y.3"]


t0=int(300)
t1=int(400)

data_solid2 = data_solid[data_solid["time"]>t0]
data_solid2 = data_solid2[data_solid2["time"]<t1]
coherence = get_coherence(data_solid)
#print(coherence)
fig = kymo_directions (data_solid2)
fig
fig.savefig("../run11_coherence2.png")

/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=10).mean()
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: FutureWarning: pd.rolling_apply is deprecated for ndarrays and will be removed in a future version
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 9600 but corresponding boolean dimension is 9599
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 9600 but corresponding boolean dimension is 9599
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: boolean index did not match inde

In [76]:
#Ouverture des fichiers
solid  = open("solid.txt")
data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
solid.close()

kt1x = data_solid["x.2"]
kt1y = data_solid["y.2"]
kt2x = data_solid["x.3"]
kt2y = data_solid["y.3"]


t0=int(0)
t1=int(500)

data_solid2 = data_solid[data_solid["time"]>t0]
data_solid2 = data_solid2[data_solid2["time"]<t1]
coherence = get_coherence(data_solid2)

unique, counts = np.unique(coherence, return_counts=True)

dict(zip(unique, counts))
name = ["N-N", "N-P", "N-AP", "AP-P", "P-P", "AP-AP"]
plt.pie(counts, startangle=90,autopct='%1.1f%%', shadow=False, labels=name)
#plt.show()
plt.savefig("../run11_proportion.png")

/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=10,center=False).mean()
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: FutureWarning: pd.rolling_apply is deprecated for ndarrays and will be removed in a future version
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 9600 but corresponding boolean dimension is 9599
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 9600 but corresponding boolean dimension is 9599
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: boolean index did not match inde

In [28]:

counts= [8,11,18,54,3,7]
name = ["N-N", "N-P", "N-AP", "AP-P", "P-P", "AP-AP"]
plt.subplots(figsize=(5,5))
plt.legend(fontsize=200)
plt.pie(counts, startangle=90,autopct='%1.1f%%', shadow=False, labels=name)
#plt.show()

plt.savefig("proportion_bio.png")

/home/sophie/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:531: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [6]:
def distance(x1, y1, x2, y2) : 
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def middle_kinetochore_position (x1, y1, x2, y2) : 
    xposition = x1 + (x2-x1)/2
    yposition = y1 + (y2-y1)/2
    return (xposition, yposition)

def start_oscillation (couple1, couple2):

    allattached1 = couple1 [ couple1["attached\\attached"]>=1 ]
    allattached2 = couple2 [ couple2["attached\\attached"]>=1 ]
    minTimeAttached1 = allattached1["time"].min()
    minTimeAttached2 = allattached2["time"].min()

    start_oscillation = max(minTimeAttached1, minTimeAttached2) 

    return start_oscillation


def end_oscillation(solid):
    dist = distance(solid["x"], solid["y"], solid["x.1"], solid["y.1"])    
    
    solid = solid.assign(dist=dist.values)
    dist = solid[solid["dist"]>=2]
    
    end_oscillation = dist["time"].min()
    
    return end_oscillation

In [8]:
os.chdir ("/media/sophie/Sophie/Resultats_simulations/20170507/result/metaphase0/")
list_run = os.listdir()
tot_count = [0,0,0,0,0,0]
for run in list_run : 
    print (run)
    
    solid  = open(run+"/solid.txt")
    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
    solid.close()

    ktmt1  = open(run+"/couple:KTMT.txt")
    ktmt2  = open(run+"/couple:KTMT2.txt")

    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
    fin_oscillation = end_oscillation(data_solid)



    t0=debut_oscillation
    t1=fin_oscillation

    data_solid2 = data_solid[data_solid["time"]>t0]
    data_solid2 = data_solid2[data_solid2["time"]<t1]
    coherence = get_coherence(data_solid2)

    unique, counts = np.unique(coherence, return_counts=True)
    for i in range(0, len(counts)):
        tot_count[i] = tot_count[i] + counts[i]
    
dict(zip(unique, tot_count))
name = ["N-N", "N-P", "N-AP", "AP-P", "P-P", "AP-AP"]
plt.subplots(figsize=(5,5))
plt.pie(tot_count, startangle=90,autopct='%1.1f%%', shadow=False, labels=name)
#plt.show()
plt.savefig("coherence.png")
    


run1


/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=10).mean()
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: FutureWarning: pd.rolling_apply is deprecated for ndarrays and will be removed in a future version
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1544 but corresponding boolean dimension is 1543
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1544 but corresponding boolean dimension is 1543
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: boolean index did not match inde

run10


/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1531 but corresponding boolean dimension is 1530
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1531 but corresponding boolean dimension is 1530
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1531 but corresponding boolean dimension is 1530
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1531 but corresponding boolean dimension is 1530


run100


...

run22
run23
run24


/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1425 but corresponding boolean dimension is 1424
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1425 but corresponding boolean dimension is 1424
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1425 but corresponding boolean dimension is 1424
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1425 but corresponding boolean dimension is 1424


run25
run26


/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1437 but corresponding boolean dimension is 1436
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1437 but corresponding boolean dimension is 1436
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1437 but corresponding boolean dimension is 1436
/home/sophie/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1437 but corresponding boolean dimension is 1436


run27


In [23]:
tot_count

[406, 1724, 1581, 7985, 1575, 1166]

In [ ]:
name = ["AP-P", "P-P"]
tot = [7985,1575]